# 🛡️ Mastering Python Exceptions: Handling Errors Gracefully

**Welcome!** This notebook dives deep into Python's error and exception handling mechanisms. Understanding how to anticipate, catch, and handle errors is crucial for writing robust, reliable, and user-friendly applications.

**Target Audience:** Python developers aiming to build resilient applications by effectively managing errors and exceptions.

**Learning Objectives:**
*   Differentiate between Syntax Errors and Runtime Exceptions.
*   Understand the `try...except...else...finally` block and its execution flow.
*   Catch specific exceptions and access exception details.
*   Learn how to raise exceptions deliberately (`raise`).
*   Explore common built-in exception types.
*   Define and use custom exception classes for domain-specific errors.
*   Apply best practices for effective exception handling in enterprise applications.
*   Identify common pitfalls and prepare for related interview questions.

## 1. Introduction: Errors Happen!

No matter how carefully we code, errors are inevitable. They can range from simple typos to unexpected runtime conditions like network failures or invalid user input. Unhandled errors will crash your Python program, often leading to a poor user experience or data corruption.

**Exception handling** is Python's mechanism for dealing with runtime errors in a controlled manner. It allows your program to:

1.  **Detect** errors when they occur.
2.  **Respond** appropriately (e.g., log the error, show a user-friendly message, try an alternative approach, perform cleanup actions).
3.  **Continue** execution (if possible and appropriate) or terminate gracefully.

**Analogy: Safety Net**

Think of your main program logic as a trapeze artist performing a routine. Exception handling is the safety net below. If the artist slips (an error occurs), the safety net (`except` block) catches them, preventing a crash. The routine might pause, the artist might reset (`finally` block actions), or the show might end early but safely, rather than resulting in disaster.

## 2. Syntax Errors vs. Exceptions

It's important to distinguish between two main categories of errors:

### 2.1 Syntax Errors (Parsing Errors)
*   Occur *before* the program starts running, when the Python interpreter parses your code.
*   Caused by violations of Python's grammar rules (e.g., typos, incorrect indentation, missing colons, mismatched parentheses).
*   Python cannot execute the code at all if a syntax error exists.
*   You **cannot** catch Syntax Errors with `try...except` because the program doesn't even start running.
*   **Fix:** Correct the code according to Python syntax rules. Linters (like `flake8`, `pylint`) and IDEs help catch these early.

In [1]:
# Example of a SyntaxError (uncomment to see the error)
# def my_function()
#    print("Hello")

# --> File "<stdin>", line 1
# -->   def my_function()
# -->                  ^
# --> SyntaxError: invalid syntax

# Another example
# print("Mismatched parenthses")

### 2.2 Exceptions (Runtime Errors)
*   Occur *during* the execution of syntactically correct code.
*   Caused by unexpected conditions or operations that fail at runtime (e.g., dividing by zero, accessing a non-existent file, trying to add incompatible types).
*   Python raises an exception object when such an error occurs.
*   If not handled (caught), an exception will terminate the program and print a traceback.
*   You **can** handle exceptions using `try...except` blocks.

In [2]:
# Example of a TypeError (Exception)
try:
    result = 10 + '5'
except TypeError as e:
    print(f"Caught a TypeError: {e}")

# Example of a ZeroDivisionError (Exception)
try:
    denominator = 0
    quotient = 100 / denominator
except ZeroDivisionError as e:
    print(f"Caught a ZeroDivisionError: {e}")

Caught a TypeError: unsupported operand type(s) for +: 'int' and 'str'
Caught a ZeroDivisionError: division by zero


## 3. Handling Exceptions: `try...except...else...finally`

The core structure for handling exceptions is the `try...except` block, which can be extended with optional `else` and `finally` clauses.

```python
try:
    # --- Code that might raise an exception ---
    # Attempt risky operations here.
    pass 
except SpecificExceptionType1 as e1:
    # --- Handle SpecificExceptionType1 ---
    # This block runs ONLY if SpecificExceptionType1 occurs in the try block.
    # 'e1' holds the exception object with details.
    pass
except (SpecificExceptionType2, SpecificExceptionType3) as e2:
    # --- Handle SpecificExceptionType2 OR SpecificExceptionType3 ---
    # This block runs if either of these occurs.
    pass
except Exception as e_general: # Catch any other standard exception
    # --- Handle any other standard exception ---
    # Catching broad 'Exception' should be done carefully.
    # Often better to catch specific types you anticipate.
    pass
except: # Bare except (Avoid!) 
    # --- Handle ANY exception, including SystemExit, KeyboardInterrupt ---
    # This is generally discouraged as it can hide critical errors.
    pass
else:
    # --- Code to run if NO exception occurred in the try block ---
    # Useful for code that depends on the success of the try block.
    pass
finally:
    # --- Code that ALWAYS runs, regardless of exceptions ---
    # Used for cleanup actions (e.g., closing files, releasing resources).
    # Runs even if there's an uncaught exception or a return/break/continue in try/except/else.
    pass
```

**Execution Flow:**

1.  **`try` Block:** Code inside `try` is executed.
2.  **No Exception:** If `try` completes without errors, the `except` blocks are skipped, the `else` block is executed (if present), and then the `finally` block is executed (if present).
3.  **Exception Occurs:** If an exception occurs in `try`, the rest of the `try` block is skipped.
4.  **`except` Matching:** Python looks for an `except` block matching the type of exception raised. The *first* matching block is executed. Subsequent `except` blocks are skipped.
5.  **No Matching `except`:** If no matching `except` block is found, the exception propagates up the call stack (potentially crashing the program if not caught elsewhere).
6.  **`else` Block:** Skipped if an exception was caught.
7.  **`finally` Block:** Executed *always*, after `try`, `except`, or `else`, even if an error occurs within an `except` or `else` block, or if a `return`, `break`, or `continue` statement is encountered.

In [3]:
import logging

# Configure basic logging for demonstration
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)

def process_data(data):
    result = None
    file = None
    try:
        logging.info("Starting data processing.")
        value = int(data) # Potential ValueError
        result = 100 / value # Potential ZeroDivisionError
        
        # Simulate file operation
        file = open("temp_log.txt", "w")
        file.write(f"Processed value: {value}\n")
        # Simulate potential error during file write (less common)
        # if value == 1: raise IOError("Disk full simulation") 
        
    except ValueError as ve:
        logging.error(f"Invalid input data type: {ve}")
        print(f"Error: Could not convert '{data}' to an integer.")
        # Optionally re-raise or return an error indicator
        # raise # Re-raises the ValueError
    except ZeroDivisionError:
        logging.error("Attempted to divide by zero.")
        print("Error: Input cannot be zero for division.")
    except (IOError, OSError) as ioe: # Catch multiple related errors
        logging.exception(f"File operation failed: {ioe}") # Log with traceback
        print(f"Error: Could not write to temporary file: {ioe}")
    except Exception as e: # Catch any other unexpected standard exception
        logging.exception(f"An unexpected error occurred: {e}") # Log with traceback
        print(f"An unexpected error occurred: {e}")
    else:
        # This runs ONLY if the try block succeeded without any exceptions
        print(f"Processing successful! Result: {result}")
        logging.info(f"Successfully processed data, result: {result}")
        # Can return the successful result here
        return result 
    finally:
        # This ALWAYS runs, good for cleanup
        print("Executing finally block: Cleaning up resources...")
        if file and not file.closed:
            print("Closing file.")
            file.close()
        else:
            print("File was not opened or already closed.")
        # Note: If an exception occurred AND wasn't caught, 
        # or was re-raised, it continues to propagate AFTER finally finishes.
        
    # If an exception was caught and not re-raised, execution continues here
    # Often returns None or an error indicator if 'else' wasn't reached
    return None 

# --- Test Cases ---
print("--- Test Case 1: Valid Input ---")
process_data("10")

print("\n--- Test Case 2: Invalid Type ---")
process_data("abc")

print("\n--- Test Case 3: Zero Division ---")
process_data("0")

# print("\n--- Test Case 4: Simulated IO Error ---")
# process_data("1") # If IOError simulation is uncommented above

INFO: Starting data processing.
INFO: Successfully processed data, result: 10.0
INFO: Starting data processing.
ERROR: Invalid input data type: invalid literal for int() with base 10: 'abc'
INFO: Starting data processing.
ERROR: Attempted to divide by zero.


--- Test Case 1: Valid Input ---
Processing successful! Result: 10.0
Executing finally block: Cleaning up resources...
Closing file.

--- Test Case 2: Invalid Type ---
Error: Could not convert 'abc' to an integer.
Executing finally block: Cleaning up resources...
File was not opened or already closed.

--- Test Case 3: Zero Division ---
Error: Input cannot be zero for division.
Executing finally block: Cleaning up resources...
File was not opened or already closed.


**Best Practice:** Catch the *most specific* exception type(s) you expect first, then potentially broader ones if necessary. Avoid bare `except:` unless you have a very specific reason (like top-level error logging before exiting) and understand the risks.

## 4. Raising Exceptions (`raise`)

Sometimes you need to signal that an error condition has occurred based on your program's logic, even if it's not one of Python's built-in runtime errors.

The `raise` statement allows you to:
1.  **Raise a new exception:** `raise ValueError("Invalid input value provided.")`
2.  **Re-raise the current exception:** (Inside an `except` block) `raise` - useful for logging an error before letting it propagate further up the call stack.
3.  **Raise a different exception from an existing one:** `raise NewException("Something went wrong") from original_exception` - preserves the original traceback as `__cause__`.

In [4]:
def calculate_discount(price: float, discount_percent: float) -> float:
    """Calculates the final price after applying a discount.
    
    Raises:
        ValueError: If price or discount_percent are invalid.
    """
    if not isinstance(price, (int, float)) or price < 0:
        # 1. Raise a new specific exception
        raise ValueError(f"Invalid price: {price}. Price must be a non-negative number.")
        
    if not isinstance(discount_percent, (int, float)) or not 0 <= discount_percent <= 100:
        raise ValueError(f"Invalid discount: {discount_percent}. Discount must be between 0 and 100.")
        
    final_price = price * (1 - discount_percent / 100)
    return final_price

# --- Using the function --- 
try:
    print(f"Discounted price (100, 10%): {calculate_discount(100, 10)}")
    # print(f"Discounted price (50, 110%): {calculate_discount(50, 110)}") # Raises ValueError
    print(f"Discounted price (-50, 10%): {calculate_discount(-50, 10)}") # Raises ValueError
except ValueError as e:
    print(f"Error calculating discount: {e}")

# --- Re-raising example --- 
def process_and_validate(value):
    try:
        result = 100 / int(value)
        print(f"Intermediate result: {result}")
        # Now apply domain-specific validation
        if result < 5:
             raise ValueError("Processed result is too small for further steps.")
        return result
    except (ValueError, ZeroDivisionError) as e:
        logging.error(f"Initial processing failed for value '{value}': {e}")
        # 2. Re-raise the original exception after logging
        raise 

print("\n--- Re-raising Example ---")
try:
    process_and_validate("50") # Raises ValueError: Processed result is too small
    # process_and_validate("0") # Raises ZeroDivisionError
except ValueError as e:
     print(f"Caught processed ValueError: {e}")
except ZeroDivisionError as e:
     print(f"Caught processed ZeroDivisionError: {e}")

# --- Raising from example --- 
class DatabaseError(Exception):
    "Custom exception for database issues."
    pass

def connect_db():
    try:
        # Simulate a low-level connection error
        raise ConnectionRefusedError("Low level socket connection failed.")
    except ConnectionRefusedError as e:
        # 3. Raise a higher-level exception, preserving the cause
        raise DatabaseError("Failed to connect to the database.") from e

print("\n--- Raising From Example ---")
try:
    connect_db()
except DatabaseError as dbe:
    print(f"Caught DatabaseError: {dbe}")
    print(f"  Cause: {dbe.__cause__}") # Access the original exception
    print(f"  Traceback available via logging.exception or traceback module")
    # logging.exception("Database connection failed") # Would show both tracebacks

ERROR: Initial processing failed for value '50': Processed result is too small for further steps.


Discounted price (100, 10%): 90.0
Error calculating discount: Invalid price: -50. Price must be a non-negative number.

--- Re-raising Example ---
Intermediate result: 2.0
Caught processed ValueError: Processed result is too small for further steps.

--- Raising From Example ---
Caught DatabaseError: Failed to connect to the database.
  Cause: Low level socket connection failed.
  Traceback available via logging.exception or traceback module


## 5. Common Built-in Exceptions

Python has many built-in exceptions. Understanding the common ones helps you write more specific `except` blocks.

*   **`Exception`:** The base class for most non-system-exiting errors. Catching this catches almost everything (but usually too broad).
*   **`AttributeError`:** Raised when an attribute reference or assignment fails (e.g., `obj.non_existent_attribute`).
*   **`ImportError`:** Raised when an `import` statement fails to find the module or a name within the module.
    *   **`ModuleNotFoundError`:** Subclass of `ImportError`, specifically when the module itself cannot be found (Python 3.6+).
*   **`IndexError`:** Raised when a sequence subscript (index) is out of range (e.g., `my_list[10]` when list has only 5 elements).
*   **`KeyError`:** Raised when a dictionary key is not found (e.g., `my_dict['non_existent_key']`).
*   **`NameError`:** Raised when a local or global name (variable) is not found (e.g., using a variable before assigning it).
*   **`FileNotFoundError`:** Raised when trying to open a file that does not exist.
*   **`OSError`:** Base class for many OS-related errors (file not found, permissions issues). `FileNotFoundError`, `PermissionError`, `ConnectionError` are subclasses.
*   **`TypeError`:** Raised when an operation or function is applied to an object of inappropriate type (e.g., `len(123)`, `'hello' + 5`).
*   **`ValueError`:** Raised when an operation or function receives an argument that has the right type but an inappropriate value (e.g., `int('abc')`, `my_list.remove('non_existent_value')`).
*   **`ZeroDivisionError`:** Raised when the second argument of a division or modulo operation is zero.
*   **`KeyboardInterrupt`:** Raised when the user hits the interrupt key (usually Ctrl+C).
*   **`SystemExit`:** Raised by the `sys.exit()` function. Not usually caught unless for specific cleanup.

Full hierarchy: [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)

## 6. Defining Custom Exceptions

**Why?** Built-in exceptions cover general error conditions. For errors specific to your application's domain logic (e.g., `InvalidTransactionError`, `UserProfileNotFoundError`), custom exceptions provide:

*   **Clarity:** Make your code more readable and self-documenting.
*   **Specific Handling:** Allow callers to catch and handle your application's specific errors differently from generic errors.
*   **Information:** Can carry additional context about the specific error.

**How?** Create a new class that inherits (directly or indirectly) from the base `Exception` class.

In [ ]:
import logging
import time

# --- Define a Hierarchy of Custom Exceptions ---

# Base exception for our application domain
class PaymentGatewayError(Exception):
    "Base exception for payment processing errors."
    def __init__(self, message="An error occurred with the payment gateway.", transaction_id=None):
        self.transaction_id = transaction_id
        # Ensure the message is passed to the base Exception class
        super().__init__(message)

# Specific error types inheriting from the base
class InsufficientFundsError(PaymentGatewayError):
    "Raised when the account has insufficient funds."
    def __init__(self, message="Insufficient funds for the transaction.", transaction_id=None, balance=None):
        super().__init__(message, transaction_id)
        self.balance = balance # Add specific context

class CardDeclinedError(PaymentGatewayError):
    "Raised when the credit card is declined."
    def __init__(self, message="The payment card was declined.", transaction_id=None, decline_code=None):
        super().__init__(message, transaction_id)
        self.decline_code = decline_code # Add specific context

class NetworkError(PaymentGatewayError):
    "Raised for network issues during payment processing."
    def __init__(self, message="A network error occurred while contacting the gateway.", transaction_id=None):
        super().__init__(message, transaction_id)

# --- Function Simulating Payment Processing ---
def process_payment(amount: float, card_token: str, current_balance: float):
    transaction_id = f"txn_{int(time.time())}" # Simple transaction ID
    print(f"Attempting payment of ${amount:.2f} with TID {transaction_id}")
    
    # Simulate different error conditions
    if amount > current_balance:
        raise InsufficientFundsError(transaction_id=transaction_id, balance=current_balance)
    
    if card_token == "declined_token":
        raise CardDeclinedError(transaction_id=transaction_id, decline_code="05-Do Not Honor")
    
    if card_token == "network_fail_token":
        # Simulate a low-level error causing our network error
        try:
            raise TimeoutError("Gateway connection timed out")
        except TimeoutError as te:
            raise NetworkError(transaction_id=transaction_id) from te
            
    if amount < 0:
        # Can raise built-in errors too
        raise ValueError("Payment amount cannot be negative.")
        
    # Simulate success
    print(f"Payment successful! Transaction ID: {transaction_id}")
    return transaction_id

# --- Handling Custom Exceptions ---
print("\n--- Handling Custom Exceptions --- ")
test_scenarios = [
    (50.0, "valid_token", 100.0),   # Success
    (150.0, "valid_token", 100.0),  # Insufficient Funds
    (75.0, "declined_token", 200.0),# Card Declined
    (25.0, "network_fail_token", 50.0),# Network Error
    (-10.0, "valid_token", 100.0)   # ValueError
]

for amount, token, balance in test_scenarios:
    try:
        process_payment(amount, token, balance)
        
    # Catch specific custom exceptions first
    except InsufficientFundsError as ife:
        logging.warning(f"Payment failed (TID: {ife.transaction_id}): {ife}. Current balance: {ife.balance}")
        print(f"Payment Warning: {ife}")
    except CardDeclinedError as cde:
        logging.error(f"Payment failed (TID: {cde.transaction_id}): {cde}. Decline code: {cde.decline_code}")
        print(f"Payment Error: {cde}")
    # Catch the custom base class to handle related errors generally
    except PaymentGatewayError as pge: 
        # This will catch NetworkError or any future subclasses we add
        logging.exception(f"Payment gateway error (TID: {pge.transaction_id}): {pge}") # Log with traceback
        print(f"Payment System Error: {pge}")
        if pge.__cause__:
            print(f"  Caused by: {pge.__cause__}")
            
    # Catch other standard errors
    except ValueError as ve:
         logging.error(f"Invalid input for payment: {ve}")
         print(f"Input Error: {ve}")
         
    print("---") # Separator between tests


--- Handling Custom Exceptions --- 


NameError: name 'time' is not defined

## 7. The `with` Statement (Context Managers)

While not strictly exception *handling*, the `with` statement is closely related because it provides a clean and robust way to manage resources (like files, network connections, locks) and ensures cleanup happens **even if exceptions occur**.

Objects used in a `with` statement are called **context managers**. They must implement two special methods:
*   `__enter__(self)`: Executed when entering the `with` block. Can return an object to be used inside the block (assigned to the `as` variable).
*   `__exit__(self, exc_type, exc_val, exc_tb)`: Executed when exiting the `with` block (either normally or due to an exception).
    *   `exc_type`, `exc_val`, `exc_tb`: Hold the exception type, value, and traceback if an exception occurred within the `with` block; otherwise, they are `None`.
    *   If `__exit__` returns `True`, the exception is suppressed (considered handled). If it returns `False` (or `None`), the exception continues to propagate after `__exit__` finishes.

**File Handling Example:**
`open()` returns a file object which is a context manager. Its `__exit__` method automatically closes the file.

In [ ]:
# --- Without 'with' (Requires manual cleanup in finally) ---
file = None
try:
    file = open("example.txt", "w")
    file.write("Hello without with\n")
    # What if an error happens here?
    # result = 1 / 0 
except IOError as e:
    print(f"IOError occurred: {e}")
finally:
    # MUST remember to close the file
    if file and not file.closed:
        print("(Manual) Closing file.")
        file.close()

# --- Using 'with' (Cleaner and safer) ---
try:
    # File is automatically closed when exiting the 'with' block,
    # even if errors occur inside.
    with open("example.txt", "a") as file: # Append mode
        file.write("Hello using with!\n")
        # If an error occurs here, file.close() is still called by __exit__
        # result = 1 / 0 
    print("File operations within 'with' completed.")
except IOError as e:
    # Catch errors related to open() or write() itself
    print(f"IOError occurred during 'with': {e}")
except ZeroDivisionError:
    print("Caught ZeroDivisionError inside 'with' block (file was still closed).")

# The 'file' variable exists outside 'with', but the file is closed
# print(f"Is file closed after 'with'? {file.closed}")

**Key Takeaway:** Use the `with` statement whenever you work with resources that need guaranteed cleanup (files, network sockets, database connections, locks) to make your code more concise and robust against errors.

## 8. Best Practices & Enterprise Considerations

1.  **Be Specific:** Catch the most specific exceptions you anticipate, rather than broad `except Exception:` or bare `except:`.
2.  **Don't Swallow Exceptions:** Avoid empty `except:` blocks or `except: pass`. If you catch an exception, handle it meaningfully (log it, inform the user, retry, clean up) or re-raise it.
3.  **Use `finally` for Cleanup:** Guarantee resource release (closing files, unlocking mutexes) in `finally` or, preferably, use context managers (`with`).
4.  **Use Custom Exceptions:** Define custom exceptions for domain-specific errors to improve code clarity and allow targeted handling.
5.  **Log Exceptions:** Always log exceptions (ideally with tracebacks using `logging.exception` or `logging.error(..., exc_info=True)`) before handling or re-raising them. This is critical for debugging in production.
6.  **Provide Context:** When raising exceptions (especially custom ones), include relevant information in the message or as attributes on the exception object.
7.  **Error Reporting:** In applications, translate exceptions into user-friendly error messages or error codes. Don't expose raw tracebacks to end-users (security risk).
8.  **Fail Fast (Often):** In many cases, it's better to let unexpected errors propagate up (and be caught by a top-level handler) than to guess how to recover deep within the code.
9.  **Consider Retries (Carefully):** For transient errors (like temporary network issues), implement a retry mechanism with backoff, but avoid infinite retries.
10. **Testing:** Write unit tests to verify that your code raises the correct exceptions under specific conditions and that your `except` blocks handle them as expected.

## 9. Pitfalls and Common Interview Questions

**Common Pitfalls:**

*   **Bare `except:`:** Catches *everything*, including `SystemExit` and `KeyboardInterrupt`, making it hard to terminate the program and hiding the real errors.
*   **`except Exception:`:** Still quite broad. Better to catch specific subclasses unless it's a generic fallback handler.
*   **Swallowing Errors (`except: pass`):** Hides problems, making debugging impossible.
*   **Incorrect `except` Order:** If a base class exception (like `Exception`) is caught *before* a more specific subclass (like `ValueError`), the specific block will never be reached.
*   **Resource Leaks:** Forgetting to close files or release resources if not using `finally` or `with`.
*   **Raising Generic `Exception`:** Prefer raising more specific built-in or custom exceptions.
*   **Performance Impact:** While necessary, `try...except` blocks do have a small performance overhead. Avoid using them excessively for non-exceptional control flow.

**Common Interview Questions:**

1.  What is the difference between a Syntax Error and an Exception in Python?
2.  Explain the purpose of the `try`, `except`, `else`, and `finally` clauses.
3.  Why is it generally bad practice to use a bare `except:`?
4.  How do you access the exception object within an `except` block?
5.  How can you catch multiple specific exceptions in a single `except` block?
6.  What does the `raise` keyword do? How can you re-raise an exception?
7.  Why would you define custom exceptions? How do you do it?
8.  What is the `with` statement used for, and how does it relate to exception handling?
9.  Name some common built-in Python exceptions and when they occur (`TypeError`, `ValueError`, `KeyError`, `IndexError`, `FileNotFoundError`).
10. How should exceptions be logged effectively?
11. What is the difference between `Exception` and `BaseException`? (BaseException is the ultimate base, includes system-exiting exceptions like KeyboardInterrupt, SystemExit. Usually catch Exception or specific subclasses).

## 10. Challenge: Robust File Parser

**Goal:** Write a function that reads data from a file, performs a simple calculation, and handles potential errors robustly.

**File Format:** Assume a file (`data.txt`) where each line contains a number (integer or float).

**Tasks:**

1.  **Create Sample Files:**
    *   `data_ok.txt`: Contains valid numbers (e.g., `10\n20.5\n-5\n`).
    *   `data_bad.txt`: Contains a mix of valid numbers and invalid lines (e.g., `10\nhello\n30\n`).
    *   Ensure one file (`data_missing.txt`) does not exist.
2.  **Define Custom Exception:** Create a custom exception `FileProcessingError` inheriting from `Exception`.
3.  **Write `parse_and_sum(filepath: str) -> float` function:**
    *   Takes the file path as input.
    *   Uses a `try...except...finally` structure.
    *   Opens and reads the file line by line **using a `with` statement**.
    *   For each line:
        *   Attempts to convert the line (strip whitespace first!) to a float.
        *   If conversion fails (`ValueError`), logs a warning with the line number and content, and skips the line.
        *   Adds the valid number to a running total.
    *   Handles `FileNotFoundError` by logging an error and raising your custom `FileProcessingError` containing an informative message and the original exception (`from e`).
    *   Handles `IOError` (or `OSError`) during file operations similarly, raising `FileProcessingError from e`.
    *   The `finally` block should log a message indicating whether processing finished or was interrupted by an error.
    *   Returns the calculated sum if successful.
4.  **Test:** Call your function with the different file paths (`data_ok.txt`, `data_bad.txt`, `data_missing.txt`) and wrap these calls in another `try...except` block to catch the potential `FileProcessingError` and print appropriate messages.

In [ ]:
# --- Solution Space for Challenge ---
import logging
import time # For simulating file creation delay if needed

# Configure logging
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    force=True)

# 1. Create Sample Files
try:
    with open("data_ok.txt", "w") as f:
        f.write("10\n")
        f.write("20.5\n")
        f.write("-5\n")
        f.write("  15  \n") # Test stripping whitespace
    with open("data_bad.txt", "w") as f:
        f.write("100\n")
        f.write("invalid_number\n")
        f.write("50.0\n")
        f.write("\n") # Test empty line
    logging.info("Sample files created.")
except IOError as e:
    logging.error(f"Failed to create sample files: {e}")

# 2. Define Custom Exception
class FileProcessingError(Exception):
    "Custom exception for errors during file parsing and processing."
    pass 

# 3. Write parsing function
def parse_and_sum(filepath: str) -> float:
    """Reads numbers from a file, sums them, handling errors.

    Args:
        filepath: Path to the data file.

    Returns:
        The sum of valid numbers in the file.

    Raises:
        FileProcessingError: If the file cannot be found or read, 
                           or a critical IO error occurs.
    """
    total_sum = 0.0
    line_num = 0
    error_occurred = False
    
    try:
        logging.info(f"Attempting to process file: {filepath}")
        # Use 'with' for automatic file closing
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line_num += 1
                cleaned_line = line.strip()
                
                if not cleaned_line: # Skip empty lines
                    continue 
                    
                try:
                    number = float(cleaned_line)
                    total_sum += number
                    logging.debug(f"Line {line_num}: Added {number}")
                except ValueError:
                    # Handle conversion errors gracefully
                    logging.warning(f"Line {line_num}: Skipping invalid number: '{cleaned_line}'")
                    # Continue to the next line
            
            # If loop completes without IO error
            logging.info(f"Finished processing file {filepath}. Total sum: {total_sum}")
            
    except FileNotFoundError as e:
        error_occurred = True
        logging.error(f"File not found: {filepath}")
        # Raise custom exception, preserving the original cause
        raise FileProcessingError(f"Input file '{filepath}' does not exist.") from e
    except OSError as e: # Catch other potential IO errors (permissions, etc.)
        error_occurred = True
        logging.exception(f"OS error reading file {filepath}: {e}") # Log with traceback
        raise FileProcessingError(f"Error reading file '{filepath}'. Check permissions or disk.") from e
    except Exception as e: # Catch any other unexpected error during processing
        error_occurred = True
        logging.exception(f"Unexpected error processing file {filepath}: {e}")
        raise FileProcessingError(f"An unexpected error occurred processing '{filepath}'.") from e
    finally:
        # Log completion status
        if error_occurred:
             logging.info(f"File processing for {filepath} was interrupted by an error.")
        else:
             # Note: This might log before the successful completion log in the try block
             # if no exception occurred. 
             logging.info(f"Finished executing try-finally for {filepath}.")
             
    # Return sum only if processing completed successfully within the try block
    # (This assumes we don't want a partial sum if an IO error happened mid-file)
    if not error_occurred:
         return total_sum
    else:
         # Should have been raised already, but as a safeguard:
         raise FileProcessingError("Processing failed, returning no sum.")

# 4. Test the function
test_files = ["data_ok.txt", "data_bad.txt", "data_missing.txt"]

print("\n--- Testing File Parser ---")
for filename in test_files:
    print(f"\n--- Processing {filename} ---")
    try:
        result_sum = parse_and_sum(filename)
        print(f"Successfully processed {filename}. Sum: {result_sum}")
    except FileProcessingError as fpe:
        print(f"Caught File Processing Error: {fpe}")
        if fpe.__cause__:
            print(f"  Original cause: {type(fpe.__cause__).__name__}: {fpe.__cause__}")
    except Exception as e:
        # Catch any unexpected errors from the calling code itself
        print(f"Caught unexpected error during test: {e}")

## 11. Conclusion

Robust error and exception handling is non-negotiable for reliable software. Python's `try...except...else...finally` structure, combined with specific exception catching, custom exception types, and the `with` statement for resource management, provides a powerful toolkit.

By anticipating potential errors, handling them gracefully, logging appropriately, and using custom exceptions for clarity, you can build applications that are more resilient, easier to debug, and provide a better experience for users even when things go wrong. Remember that exception handling is not just about preventing crashes; it's about managing failure in a controlled and informative way.